In [27]:
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [46]:
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertConfig, BertModel
from transformers.models.bert.modeling_bert import BertEncoder
import numpy as np
import pandas as pd
from sklearn import metrics
import transformers
import torch
import k_diffusion as K

In [56]:
config = BertConfig(hidden_size=1024, num_attention_heads=16)
bert_model = BertModel(config)
model = torch.nn.Sequential(*list(bert_model.children())[1:])
# encoder = BertEncoder(config)

In [34]:
import k_diffusion as K
ds = K.datasets.ShardedTensorDataset("/shared/amyxlu/data/cath/shards/seqlen_256")
dl = torch.utils.data.DataLoader(ds, batch_size=4)
batch = next(iter(dl))

In [74]:
model = K.models.ProteinBertDenoiser()

In [75]:
model = model.to(device)
model = model.to(torch.bfloat16)

In [76]:
import einops

x = batch[0].to(device)
mask = (
    einops.repeat(
        torch.arange(x.shape[1])[None, :],
        "1 L -> N L",
        N=x.shape[0],
    )
    < batch[1][:, None]
).to(device)

In [77]:
sigma = torch.randn(x.shape[0]).to(device).to(dtype=torch.bfloat16)
output = model(x, sigma, mask)